In [ ]:
pip install openmeteo-requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.1/167.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.2/683.2 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.8/145.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.1/394.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.1 MB/s eta 0:00:00


In [ ]:
pip install requests-cache retry-requests numpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4/61.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.7/70.7 kB 4.9 MB/s eta 0:00:00


BANGKOK SAVING

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 13.754,
	"longitude": 100.5014,
	"hourly": ["pm10", "pm2_5", "ammonia", "carbon_monoxide", "nitrogen_dioxide", "carbon_dioxide", "sulphur_dioxide", "methane"],
	"current": "ozone",
	"timezone": "GMT",
	"start_date": "2020-12-01",
	"end_date": "2025-12-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process current data. The order of variables needs to be the same as requested.
current = response.Current()
current_ozone = current.Variables(0).Value()

print(f"\nCurrent time: {current.Time()}")
print(f"Current ozone: {current_ozone}")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(5).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(6).ValuesAsNumpy()
hourly_methane = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["ammonia"] = hourly_ammonia
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["methane"] = hourly_methane

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: 13.800003051757812°N 100.5°E
Elevation: 7.0 m asl
Timezone: NoneNone
Timezone difference to GMT+0: 0s

Current time: 1765191600
Current ozone: 72.0

Hourly data
                            date       pm10      pm2_5  ammonia  \
0     2020-12-01 00:00:00+00:00        NaN        NaN      NaN   
1     2020-12-01 01:00:00+00:00        NaN        NaN      NaN   
2     2020-12-01 02:00:00+00:00        NaN        NaN      NaN   
3     2020-12-01 03:00:00+00:00        NaN        NaN      NaN   
4     2020-12-01 04:00:00+00:00        NaN        NaN      NaN   
...                         ...        ...        ...      ...   
43843 2025-12-01 19:00:00+00:00  28.299999  28.299999      NaN   
43844 2025-12-01 20:00:00+00:00  27.200001  27.200001      NaN   
43845 2025-12-01 21:00:00+00:00  26.299999  26.299999      NaN   
43846 2025-12-01 22:00:00+00:00  25.700001  25.600000      NaN   
43847 2025-12-01 23:00:00+00:00  25.100000  25.100000      NaN   

       carbon_monoxide  nitrogen

In [ ]:
hourly_dataframe.to_csv('Bangkok.csv', index=False)
print('Dataset saved to Bangkok.csv')

Dataset saved to Bangkok.csv


JAKARTA SAVING

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": -6.1818,
	"longitude": 106.8223,
	"hourly": ["pm10", "pm2_5", "ammonia", "carbon_monoxide", "nitrogen_dioxide", "carbon_dioxide", "sulphur_dioxide", "methane"],
	"current": "ozone",
	"timezone": "GMT",
	"start_date": "2020-12-01",
	"end_date": "2025-12-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process current data. The order of variables needs to be the same as requested.
current = response.Current()
current_ozone = current.Variables(0).Value()

print(f"\nCurrent time: {current.Time()}")
print(f"Current ozone: {current_ozone}")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(5).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(6).ValuesAsNumpy()
hourly_methane = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["ammonia"] = hourly_ammonia
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["methane"] = hourly_methane

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: -6.1999969482421875°N 106.80001831054688°E
Elevation: 15.0 m asl
Timezone: NoneNone
Timezone difference to GMT+0: 0s

Current time: 1765191600
Current ozone: 25.0

Hourly data
                            date        pm10       pm2_5  ammonia  \
0     2020-12-01 00:00:00+00:00         NaN         NaN      NaN   
1     2020-12-01 01:00:00+00:00         NaN         NaN      NaN   
2     2020-12-01 02:00:00+00:00         NaN         NaN      NaN   
3     2020-12-01 03:00:00+00:00         NaN         NaN      NaN   
4     2020-12-01 04:00:00+00:00         NaN         NaN      NaN   
...                         ...         ...         ...      ...   
43843 2025-12-01 19:00:00+00:00   97.300003   97.099998      NaN   
43844 2025-12-01 20:00:00+00:00   96.199997   95.699997      NaN   
43845 2025-12-01 21:00:00+00:00   99.400002   99.000000      NaN   
43846 2025-12-01 22:00:00+00:00  108.699997  108.300003      NaN   
43847 2025-12-01 23:00:00+00:00  128.100006  127.599998      N

In [ ]:
hourly_dataframe.to_csv('Jakarta.csv', index=False)
print('Dataset saved to Jakarta.csv')

Dataset saved to Jakarta.csv


TOKYO SAVING

In [ ]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://air-quality-api.open-meteo.com/v1/air-quality"
params = {
	"latitude": 35.6895,
	"longitude": 139.6917,
	"hourly": ["pm10", "pm2_5", "ammonia", "carbon_monoxide", "nitrogen_dioxide", "carbon_dioxide", "sulphur_dioxide", "methane"],
	"current": "ozone",
	"timezone": "GMT",
	"start_date": "2020-12-01",
	"end_date": "2025-12-01",
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates: {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation: {response.Elevation()} m asl")
print(f"Timezone: {response.Timezone()}{response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0: {response.UtcOffsetSeconds()}s")

# Process current data. The order of variables needs to be the same as requested.
current = response.Current()
current_ozone = current.Variables(0).Value()

print(f"\nCurrent time: {current.Time()}")
print(f"Current ozone: {current_ozone}")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_pm10 = hourly.Variables(0).ValuesAsNumpy()
hourly_pm2_5 = hourly.Variables(1).ValuesAsNumpy()
hourly_ammonia = hourly.Variables(2).ValuesAsNumpy()
hourly_carbon_monoxide = hourly.Variables(3).ValuesAsNumpy()
hourly_nitrogen_dioxide = hourly.Variables(4).ValuesAsNumpy()
hourly_carbon_dioxide = hourly.Variables(5).ValuesAsNumpy()
hourly_sulphur_dioxide = hourly.Variables(6).ValuesAsNumpy()
hourly_methane = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end =  pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["pm10"] = hourly_pm10
hourly_data["pm2_5"] = hourly_pm2_5
hourly_data["ammonia"] = hourly_ammonia
hourly_data["carbon_monoxide"] = hourly_carbon_monoxide
hourly_data["nitrogen_dioxide"] = hourly_nitrogen_dioxide
hourly_data["carbon_dioxide"] = hourly_carbon_dioxide
hourly_data["sulphur_dioxide"] = hourly_sulphur_dioxide
hourly_data["methane"] = hourly_methane

hourly_dataframe = pd.DataFrame(data = hourly_data)
print("\nHourly data\n", hourly_dataframe)

Coordinates: 35.70000457763672°N 139.70001220703125°E
Elevation: 40.0 m asl
Timezone: NoneNone
Timezone difference to GMT+0: 0s

Current time: 1765191600
Current ozone: 1.0

Hourly data
                            date       pm10      pm2_5  ammonia  \
0     2020-12-01 00:00:00+00:00        NaN        NaN      NaN   
1     2020-12-01 01:00:00+00:00        NaN        NaN      NaN   
2     2020-12-01 02:00:00+00:00        NaN        NaN      NaN   
3     2020-12-01 03:00:00+00:00        NaN        NaN      NaN   
4     2020-12-01 04:00:00+00:00        NaN        NaN      NaN   
...                         ...        ...        ...      ...   
43843 2025-12-01 19:00:00+00:00  39.599998  38.900002      NaN   
43844 2025-12-01 20:00:00+00:00  36.700001  35.900002      NaN   
43845 2025-12-01 21:00:00+00:00  38.400002  37.700001      NaN   
43846 2025-12-01 22:00:00+00:00  42.799999  42.099998      NaN   
43847 2025-12-01 23:00:00+00:00  42.700001  41.700001      NaN   

       carbon_monoxi

In [ ]:
hourly_dataframe.to_csv('Tokyo.csv', index=False)
print('Dataset saved to Tokyo.csv')

Dataset saved to Tokyo.csv
